In [21]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
import csv
from getpass import getpass
import pandas as pd 
import dash
import dash_core_components as dcc
import dash_html_components as html
import dash_table
import dash_renderer
from dash.dependencies import Input, Output, State
import re
import sys
import os

In [27]:
def get_tweet_info(tweet):
    
    #Extract data from tweet element
    
    #since username is the first span in the tree using the simple way
    username = tweet.find_element_by_xpath('.//span').text
    postdate = tweet.find_element_by_xpath('.//time').get_attribute('datetime')
    
    #grab first  2 divs seperately and concatenate them for the text
    comment = tweet.find_element_by_xpath('.//div[2]/div[2]/div[1]').text
    responding = tweet.find_element_by_xpath('.//div[2]/div[2]/div[2]').text
    text = comment + responding
    #fetching reply , retweet and like count bt just simple data-testid attribute
    reply_cnt = tweet.find_element_by_xpath('.//div[@data-testid="reply"]').text
    retweet_cnt = tweet.find_element_by_xpath('.//div[@data-testid="retweet"]').text
    like_cnt = tweet.find_element_by_xpath('.//div[@data-testid="like"]').text
    
    tweet = (username, postdate, text,reply_cnt, retweet_cnt, like_cnt)
    return tweet

In [33]:
# application variables

user = input('username: ')
my_password = getpass('Password: ')

'''
user = "@aysel25377503"
my_password = "Ax4567.l!W"
'''



# create instance of web driver
browser = webdriver.Firefox()

# navigate to login screen
browser.get('https://www.twitter.com/login')

#login
username = browser.find_element_by_xpath('//input[@name="session[username_or_email]"]')
username.send_keys(user)

password = browser.find_element_by_xpath('//input[@name="session[password]"]')
password.send_keys(my_password)
password.send_keys(Keys.RETURN)
sleep(5)

# find search input and search for term
search_input = browser.find_element_by_xpath('//input[@aria-label="Search query"]')
search_input.send_keys("#requestforstartup")
search_input.send_keys(Keys.RETURN)
sleep(2)




username: @aysel25377503
Password: ········


In [34]:
data = []
tweet_ids = set() #preventing the duplicate tweets 
last_position = browser.execute_script("return window.pageYOffset;")
scrolling = True

while scrolling:
    page_cards = browser.find_elements_by_xpath('//div[@data-testid="tweet"]')
    for card in page_cards[-15:]:#we dont have to check all tweets while scrolling long process
        tweet = get_tweet_info(card)
        if tweet:
            tweet_id = ''.join(tweet)
            if tweet_id not in tweet_ids:
                tweet_ids.add(tweet_id)
                data.append(tweet)
            
    scroll_attempt = 0
    while True:
        # check scroll position
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight);')
        sleep(2) #wait for the page load
        curr_position = browser.execute_script("return window.pageYOffset;")
        if last_position == curr_position:
            scroll_attempt += 1
            
            # end of scroll region
            if scroll_attempt >= 3:
                scrolling = False
                break
            else:
                sleep(2) # attempt another scroll
        else:
            last_position = curr_position
            break

# close the web driver
browser.close()


In [17]:
#saving data in a csv file
with open('allTweets.csv', 'w', newline='', encoding='utf-8') as f:
    header =['Username', 'postdate', 'text','comments','Retweets','Likes']
    writer = csv.writer(f)
    writer.writerow(header)
    writer.writerows(data)



In [2]:
#creating a dataframe from csv 
data =pd.read_csv('allTweets.csv')
df = pd.DataFrame(data,
                   columns=['Username', 'postdate', 'text','comments','Retweets','Likes'])
#filling NaN values with 0
df = df.fillna(0)

#converting "1K or 1M" values to integer 
df['comments'] = df['comments'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
#converting entire column to integer to use with sort function correctly
df['comments'] = df['comments'].astype(int)

#Retweets
df['Retweets'] = df['Retweets'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
df['Retweets'] = df['Retweets'].astype(int)

#likes 
df['Likes'] = df['Likes'].replace({'K': '*1e3', 'M': '*1e6'}, regex=True).map(pd.eval).astype(int)
df['Likes'] = df['Likes'].astype(int)

#sorting  df by multiple keys comment , retweets and likes
df = df.sort_values(['comments','Likes','Retweets'], ascending=False)
df.head(10)

,Username,postdate,text,comments,Retweets,Likes
54,Elon Musk,2019-10-12T20:32:25.000Z,French fries \n are the devil,1100,7500,74600
29,Paul Graham,2020-05-11T14:53:51.000Z,Most colleges are planning to teach in person ...,80,129,766
31,Kelly Vaughn,2020-05-24T22:41:27.000Z,"""I wish I had more time to read!"" she says as ...",28,75,928
0,Andreas Klinger,2020-01-25T21:00:15.000Z,Remote Work is one of the big opportunities fo...,17,63,270
37,daniel debow,2020-08-10T16:08:27.000Z,If Leo had started Fender Guitars today... it ...,6,1,28
3,Kieran | nocodelife,2020-06-10T11:59:52.000Z,Product idea: Asynchronous team voice chat app...,5,0,10
5,Sharath Kuruganty,2020-10-08T20:49:45.000Z,Is there a \n@sandwich\n video for maker produ...,4,1,6
7,Minn,2020-03-10T18:54:41.000Z,Perhaps now is the time for what \n@jamescham\...,3,1,8
10,Minn,2020-03-09T05:37:45.000Z,"Pair programming, but for knowledge work \n\nT...",3,1,5
11,Shawn Cheng,2019-09-09T17:12:02.000Z,Can someone build a \n@gyroscope_app\n for bab...,3,0,3


# Displaying data via interface

In [ ]:
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets=external_stylesheets)



app.layout = html.Div([
    html.Div([
       
                   dcc.Markdown('''
              > This  software searches Twitter for the term **request for startup** and extracts relevant information about tweets,
              >such as author, date, tweet contents, number of likes, number of retweets and number of discussions.
              > **Tweets are sorted by total number of retweets, likes, and discussions.**
              >Created by Aysel Havutcu.
         ''')  
    ],style={'background':' #c4e3ed'}),

    dash_table.DataTable(
     style_data={
        'whiteSpace': 'normal',
        'height': 'auto',
        'lineHeight': '15px'
    },
    data=df.to_dict('records'),
    columns=[{'id': c, 'name': c} for c in df.columns],
    style_table={'minWidth': '100%','height': 500,'overflowY': 'auto'},
    css=[{
        'selector': '.dash-spreadsheet td div',
        'rule': '''
            line-height: 15px;
            max-height: 30px; min-height: 30px; height: 30px;
            display: block;
            overflow-y: hidden;
        '''
    }],
    
    tooltip_data=[#the tooltip will persist as long as the mouse pointer is above the cell, and it will disappear when the pointer moves away.
        {
            column: {'value': str(value), 'type': 'markdown'}
            for column, value in row.items()
        } for row in df.to_dict('rows')
    ],
    tooltip_duration=None,
    #styling the table
    style_cell={'textAlign': 'left'}, # left align text in columns for readability
     style_cell_conditional=[
        {
            'if': {'column_id': c},
            'textAlign': 'left'
        } for c in ['Date', 'Region']
    ],
    style_data_conditional=[
        {
            'if': {'row_index': 'odd'},
            'backgroundColor': 'rgb(248, 248, 248)'
        }
    ],
    style_header={
        'backgroundColor': 'rgb(230, 230, 230)',
        'fontWeight': 'bold'
    }
)

])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [18/Dec/2020 15:35:42] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2020 15:35:43] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [18/Dec/2020 15:35:43] "GET /_dash-layout HTTP/1.1" 200 -
